In [25]:
import cv2 
import numpy as np

video_path = "/Users/elhidze/Gagarin/3.mp4"

In [26]:
def extract_lighting_change(frame1, frame2):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray1, gray2)
    return np.mean(diff)

def extract_blur(frame):
    #фильтр обнаружения краев, если дисперсии мало - есть размытие
    blur = cv2.Laplacian(frame, cv2.CV_64F).var()
    return blur

def extract_camera_angle_change(frame1, frame2):
    sift = cv2.SIFT_create()
    # SIFT определяет ключевые точки и их дескрипторы
    kp1, des1 = sift.detectAndCompute(frame1, None)
    kp2, des2 = sift.detectAndCompute(frame2, None)

    # проходимся knn матчером по соответствующим точкам обоих кадров
    # если "хороших" совпадений мало - камеру сдвинули
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    return len(good_matches)

def extract_features_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    prev_frame = None
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("End of video")
            break
        
        if prev_frame is not None:
            lighting_change = extract_lighting_change(prev_frame, frame)
            blur = extract_blur(frame)
            camera_angle_change = extract_camera_angle_change(prev_frame, frame)
            
            print("Lighting Change:", lighting_change)
            print("Blur:", blur)
            print("Camera Angle Change:", camera_angle_change)
        
        prev_frame = frame
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()


In [27]:
extract_features_from_video(video_path)

Lighting Change: 0.25059444444444445
Blur: 323.88673817404333
Camera Angle Change: 1882
Lighting Change: 0.20979166666666665
Blur: 324.94613255888123
Camera Angle Change: 1920
Lighting Change: 0.1870527777777778
Blur: 325.3215385170061
Camera Angle Change: 1943
Lighting Change: 0.15370555555555557
Blur: 325.9890646785184
Camera Angle Change: 1960
Lighting Change: 0.15832222222222223
Blur: 326.85684025876526
Camera Angle Change: 1985
Lighting Change: 0.14723055555555556
Blur: 327.92643914351083
Camera Angle Change: 1994
Lighting Change: 0.14652222222222222
Blur: 328.2145186925617
Camera Angle Change: 1983
Lighting Change: 0.14832222222222222
Blur: 328.3996928840665
Camera Angle Change: 1976
Lighting Change: 0.15776666666666667
Blur: 329.1358753114122
Camera Angle Change: 1980
Lighting Change: 0.13758333333333334
Blur: 329.7260422431405
Camera Angle Change: 2004
Lighting Change: 0.20005555555555554
Blur: 329.6443663886958
Camera Angle Change: 1993
Lighting Change: 0.1361638888888889
Blur

KeyboardInterrupt: 